# Black Scholes Model Deep Calibration

Kurzer Versuch....

In [ ]:
# Libraries laden
import numpy as np
from math import log, sqrt, exp
from scipy import stats
import pandas as pd
#import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [82]:
# Function that equals R's expand_grid
import itertools
def expand_grid(data_dict):
    rows = itertools.product(*data_dict.values())
    return pd.DataFrame.from_records(rows, columns=data_dict.keys())

In [94]:
# Generate sznthetic data
synthetic_data = expand_grid({
    'stock_price': np.arange(40, 61),
    'strike_price':np.arange(20, 90),
    'maturity': np.arange(3/12, 2, step = 1/12),
    'risk_free_rate':np.arange(0.01, 0.06, step = 0.01),
    'sigma': np.arange(0.1, 0.9, step = 0.1)
}
)

In [84]:
# Black-Scholes Modell zur Berechnung der IV
def bs_option_value(S0, K, T, r, sigma):
    'Function to calculate the value of a call option based on the Black-Scholes formula'
    'arguments:'
    'S0: inital stock price'
    'K: strike price'
    'T: maturity'
    'r: risk free rate (constant)'
    'sigma: volatility'
    'returns: option value'
    #calculate d1 and d2
    d1 = ((np.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T)))
    d2 = ((np.log(S0 / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T)))
    option_value = S0 * stats.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * stats.norm.cdf(d2, 0.0, 1.0)
    return option_value

In [85]:
# Testing the function
bs_option_value(1,2,4,3,5)

0.9999999988919036

In [86]:
# calculate option vega
def option_vega(S0, K, T, r, sigma):
    'takes the same arguments as the function above'
    'returns option vega'
    d1 = ((np.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T)))
    vega = S0 * stats.norm.cdf(d1, 0.0, 1.0) * np.sqrt(T)
    return vega

In [87]:
# Testing the function
option_vega(1,2,3,4,5)

1.7320507924724615

In [88]:
# calculate implied volatility
def implied_volatility(S0, K, T, C, r, sigma):
    'calculates the implied volatility'
    'C0: '
    'sigma_est:'
    'it: number of iterations'
    'retunts implied volatility'
    d1 = (np.log(S0 / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S0 / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    fx = S0 * stats.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * stats.norm.cdf(d2, 0.0, 1.0) - C
    
    vega = (1 / np.sqrt(2 * np.pi)) * S0 * np.sqrt(T) * np.exp(-(stats.norm.cdf(d1, 0.0, 1.0) ** 2) * 0.5)
    # Warum hier andere Formal fuer vega?
    
    tolerance = 0.000001
    x0 = sigma
    xnew  = x0
    xold = x0 - 1
        
    while abs(xnew - xold) > tolerance:
    
        xold = xnew
        xnew = (xnew - fx - C) / vega
        
        return abs(xnew)
# Quelle fuer vega und iv : https://aaronschlegel.me/implied-volatility-functions-python.html

In [89]:
# Testing the function
implied_volatility(1,2,3,4,5,6)

8.683118302572918

In [95]:
synthetic_data['black_scholes'] = bs_option_value(synthetic_data['stock_price'], synthetic_data['strike_price'],
                                               synthetic_data['maturity'], synthetic_data['risk_free_rate'],
                                               synthetic_data['sigma'])

In [96]:
# Add some random noise to the real prices to create the option prices
synthetic_data["option_price"] = synthetic_data["black_scholes"] + np.random.normal(0, 0.1)

In [97]:
synthetic_data.head()

,stock_price,strike_price,maturity,risk_free_rate,sigma,black_scholes,option_price
0,40,20,0.25,0.01,0.1,20.049938,20.105943
1,40,20,0.25,0.01,0.2,20.049938,20.105943
2,40,20,0.25,0.01,0.3,20.049939,20.105945
3,40,20,0.25,0.01,0.4,20.050296,20.106302
4,40,20,0.25,0.01,0.5,20.055598,20.111603


In [173]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(synthetic_data.drop(["black_scholes"], axis = 1), test_size = 0.01,
                                                   random_state = 42)

In [101]:
# Daten standardisieren
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [157]:
def standardize(train, test, cols):
    
    # Pruefen, ob korrekt spaltenweise standardisiert wurde - > passt
    cols = cols
    
    scaler = StandardScaler().fit(train[cols])
    train_std=pd.DataFrame(scaler.transform(train[cols]), columns = cols)

    test_std=pd.DataFrame(scaler.transform(test[cols]), columns = cols)
    return train_std, test_std

In [174]:
columns = ["stock_price", 'strike_price', 'maturity', 'risk_free_rate', 'sigma', 'option_price']
train_std, test_std = standardize(train, test, columns)

In [175]:
# All variables now have 0 mean and std.dev 1
train_std.describe()

,stock_price,strike_price,maturity,risk_free_rate,sigma,option_price
count,1.222452e+06,1.222452e+06,1.222452e+06,1.222452e+06,1.222452e+06,1.222452e+06
mean,1.297920e-15,-1.399374e-16,-1.112979e-15,-2.103291e-15,1.401682e-15,-1.420584e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.651488e+00,-1.707533e+00,-1.651447e+00,-1.414344e+00,-1.527611e+00,-1.157175e+00
25%,-8.257595e-01,-8.661821e-01,-8.257010e-01,-7.072452e-01,-6.547270e-01,-8.881225e-01
50%,-3.134175e-05,2.466045e-02,4.512230e-05,-1.463419e-04,2.181567e-01,-2.134170e-01
75%,8.256968e-01,8.660118e-01,8.257912e-01,7.069525e-01,1.091040e+00,6.910842e-01
max,1.651425e+00,1.707363e+00,1.651537e+00,1.414051e+00,1.527482e+00,3.264956e+00


In [179]:
# Dataframes with Xs and ys
X_train = train_std.drop(['option_price'], axis = 1)
X_test = test_std.drop(['option_price'], axis = 1)
y_train = train['option_price']
y_test = test['option_price']

In [18]:
# Netz bauen, noch kein finetuning
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape = (5,)), # Anzahl Features
    tf.keras.layers.Dense(units = 1000, activation = 'elu'),
    tf.keras.layers.Dropout(rate = 0.25),
    tf.keras.layers.Dense(units = 800, activation = 'elu'),
    tf.keras.layers.Dropout(rate = 0.25),
    tf.keras.layers.Dense(units = 600, activation = 'elu'),
    tf.keras.layers.Dropout(rate = 0.25),
    tf.keras.layers.Dense(units = 64, activation = 'elu'),
    tf.keras.layers.Dropout(rate = 0.25),
    tf.keras.layers.Dense(units = 1, activation = 'elu')
])
model.compile(loss='mse',optimizer='adam')

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1000)              6000      
_________________________________________________________________
dropout (Dropout)            (None, 1000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 800)               800800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 800)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 600)               480600    
_________________________________________________________________
dropout_2 (Dropout)          (None, 600)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                3

In [21]:
history = model.fit(X_train_std, y_train, validation_split=0.1, batch_size = 200, epochs = 3)
# missing callback

Epoch 1/3
14029/14029 [==============================] - 356s 25ms/step - loss: 1.8503 - val_loss: 0.1151
Epoch 2/3
14029/14029 [==============================] - 404s 29ms/step - loss: 1.4303 - val_loss: 0.2295
Epoch 3/3
14029/14029 [==============================] - 417s 30ms/step - loss: 1.2823 - val_loss: 0.1271


In [30]:
y_hat = model.predict(x_test)
#y_hat = np.squeeze(y_hat)

In [31]:
y_hat

array([[-1.],
       [-1.],
       [-1.],
       ...,
       [-1.],
       [-1.],
       [-1.]], dtype=float32)

In [32]:
def CheckAccuracy(y,y_hat):
    stats = dict()
    
    stats['diff'] = y - y_hat
    
    stats['mse'] = np.mean(stats['diff']**2)
    print("Mean Squared Error:      ", stats['mse'])
    
    stats['rmse'] = np.sqrt(stats['mse'])
    print("Root Mean Squared Error: ", stats['rmse'])
    
    stats['mae'] = np.mean(abs(stats['diff']))
    print("Mean Absolute Error:     ", stats['mae'])
    
    stats['mpe'] = np.sqrt(stats['mse'])/np.mean(y)
    print("Mean Percent Error:      ", stats['mpe'])
    
    #plots
    plt.rcParams['agg.path.chunksize'] = 100000
    plt.figure(figsize=(14,10))
    plt.scatter(y, y_hat,color='black',linewidth=0.3,alpha=0.4, s=0.5)
    plt.xlabel('Actual Price',fontsize=20,fontname='Times New Roman')
    plt.ylabel('Predicted Price',fontsize=20,fontname='Times New Roman') 
    plt.show()
    
    plt.figure(figsize=(14,10))
    plt.hist(stats['diff'], bins=50,edgecolor='black',color='white')
    plt.xlabel('Diff')
    plt.ylabel('Density')
    plt.show()
    
    return stats

In [ ]:
CheckAccuracy(y_test,y_hat)